# Data Processing for UI

 1. grid_id (Grid ID)
 2. central_lat (Central Latitude)
 3. central_lon (Central Longitude)
 4. total_event_count (Total Event Count)
 5. total_flood_event (Total Flood Events)
 6. earliest_event_year (Earliest Event Year)
 7. latest_event_year (Latest Event Year)
 8. total_damages (Total Damages)
 9. primary_admin1 (Primary Administrative Region)
 10. event_summary (Event Summary)

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
from google.colab import drive
drive.mount('/content/drive/')

%cd drive/My Drive/mml_flood/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/mml_flood/'
/content/drive/My Drive/mml_flood


### Read data

In [26]:
#gdis data - geocoded disasters
gdis = pd.read_csv('data/disaster/pend-gdis-1960-2018-disasterlocations.csv')
# #emdat data - international disasters
emdat = pd.read_csv('data/disaster/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])

<ipython-input-26-53a980b4090f>:4: DtypeWarning: Columns (8,16,17,18,19,24,25,26,27,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  emdat = pd.read_csv('data/disaster/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])


  ### Filter emdat columns

In [27]:
print(emdat.columns)

Index(['Dis No', 'Year', 'Seq', 'Glide', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype',
       'Event Name', 'Country', 'ISO', 'Region', 'Continent', 'Location',
       'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response',
       'Appeal', 'Declaration', 'AID Contribution ('000 US$)', 'Dis Mag Value',
       'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time', 'River Basin',
       'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damages ('000 US$)', 'Insured Damages, Adjusted ('000 US$)',
       'Total Damages ('000 US$)', 'Total Damages, Adjusted ('000 US$)', 'CPI',
       'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Geo Locations'],
      dtype='object')


In [30]:
emdat['disasterno'] = emdat['Dis No'].str[:-4]  # Remove last 4 characters "-XYZ" area code

In [31]:
emdat_cols = ['disasterno', 'Year', 'Event Name',
              'Start Year', 'Start Month', 'Start Day',
              'End Year', 'End Month', 'End Day',
              "Total Damages, Adjusted ('000 US$)",
              'Total Deaths', 'Total Affected',
              "Reconstruction Costs, Adjusted ('000 US$)",
              'Disaster Subtype', 'OFDA Response', 'River Basin']

# Subset EMDAT to available columns (ignore missing ones if not present)
emdat = emdat[[col for col in emdat_cols if col in emdat.columns]]
print(emdat['OFDA Response'].count())  # check available data
print(emdat['River Basin'].count())    # check available data

1716
1312


In [32]:
print(emdat.columns)

Index(['disasterno', 'Year', 'Event Name', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Damages, Adjusted ('000 US$)', 'Total Deaths', 'Total Affected',
       'Reconstruction Costs, Adjusted ('000 US$)', 'Disaster Subtype',
       'OFDA Response', 'River Basin'],
      dtype='object')


In [33]:
print(gdis.columns)

Index(['id', 'country', 'iso3', 'gwno', 'year', 'geo_id', 'geolocation',
       'level', 'adm1', 'adm2', 'adm3', 'location', 'historical',
       'hist_country', 'disastertype', 'disasterno', 'latitude', 'longitude'],
      dtype='object')


### Merged cleaned emdat with gdis on disasterno

In [34]:
merged_df = pd.merge(emdat, gdis, on='disasterno', how='right')
merged_df.shape

(82885, 33)

In [35]:
# Drop duplicate records based on GDIS 'id'
merged_df = merged_df.drop_duplicates(subset=['id'])
print("Merged Data Shape:", merged_df.shape)

Merged Data Shape: (9924, 33)


In [36]:
# Create grid_id from GDIS coordinates
merged_df['lat'] = merged_df['latitude'].round().astype(int)
merged_df['lon'] = merged_df['longitude'].round().astype(int)
merged_df['grid_id'] = merged_df['lat'].astype(str) + "_" + merged_df['lon'].astype(str)


### Create a look up table including all disasters over all locations, lat and lon, location names

In [37]:
#create a table for all locations for all disasters
df_all = merged_df.copy()

# Aggregate all params
agg_all = df_all.groupby('grid_id').agg(
    all_count = ('disasterno', 'count'), #count of all disasters
    all_damages = ("Total Damages, Adjusted ('000 US$)", 'sum'),
    lat = ('lat', 'first'),
    lon = ('lon', 'first'),
    primary_admin1 = ('adm1', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
    unique_locations = ('location', lambda x: ', '.join(sorted(x.unique()))),
    all_deaths = ('Total Deaths', 'sum') if 'Total Deaths' in df_all.columns else (lambda x: np.nan),
    predominant_disaster = ('disastertype', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan) if 'disastertype' in df_all.columns else (lambda x: np.nan),
    all_disastertypes = ('disastertype', lambda x: ', '.join(sorted(x.unique()))),
    # flood_count = ()
).reset_index()

In [38]:
agg_all
# Apply lambda function with proper f-string formatting
agg_all['event_summary'] = agg_all.apply(
    lambda row: f"""In total {row['all_count']} events across all disasters;
    predominant disaster: {row['predominant_disaster'] if not pd.isna(row['predominant_disaster']) else 'N/A'};
    types of disasters: {row['all_disastertypes'] if not pd.isna(row['all_disastertypes']) else 'N/A'};
    total damages: {row['all_damages'] if not pd.isna(row['all_damages']) else 'N/A'} ('000 US$);
    total deaths: {row['all_deaths'] if not pd.isna(row['all_deaths']) else 'N/A'}.""",
    axis=1
)
agg_all
# select_cols = ['grid_id','lat','lon','primary_admin1','unique_locations','predominant_disaster','all_disaster','total_damages','total_deaths','total_disaster_count']
# agg_all = agg_all[select_cols]
# save to CSV
# agg_all.to_csv('lookup.csv')

,grid_id,all_count,all_damages,lat,lon,primary_admin1,unique_locations,all_deaths,predominant_disaster,all_disastertypes,event_summary
0,-10_-36,1,531.0,-10,-36,Alagoas,Maceio,28.0,flood,flood,In total 1 events across all disasters;\n p...
1,-10_-37,6,895928.0,-10,-37,Alagoas,Alagoas,511.0,flood,"flood, landslide",In total 6 events across all disasters;\n p...
2,-10_-48,1,0.0,-10,-48,Tocantins,Tocantins,0.0,drought,drought,In total 1 events across all disasters;\n p...
3,-10_-66,1,0.0,-10,-66,Pando,Nueva Esperanza,25.0,flood,flood,In total 1 events across all disasters;\n p...
4,-10_-75,2,0.0,-10,-75,Pasco,Pasco,418.0,extreme temperature,extreme temperature,In total 2 events across all disasters;\n p...
...,...,...,...,...,...,...,...,...,...,...,...
2822,9_78,1,0.0,9,78,Tamil Nadu,Virudhunagar district,8.0,flood,flood,In total 1 events across all disasters;\n p...
2823,9_79,2,0.0,9,79,Tamil Nadu,"Ramanathapurum, Rameswaram",507.0,storm,storm,In total 2 events across all disasters;\n p...
2824,9_80,3,1354808.0,9,80,Kilinochchi,"Vavuniya, Kilinochchi district , Northern",212.0,flood,flood,In total 3 events across all disasters;\n p...
2825,9_81,4,325863.0,9,81,Trincomalee,"Mullaitivu district, Trincomalee",249.0,flood,"flood, storm",In total 4 events across all disasters;\n p...


### Next we focus on one disaster type  

In [39]:
disaster = 'flood' #changed the code here so we could generate detailed analysis for all disasters

#subset to flooding only
df = merged_df[merged_df['disastertype'].str.lower() == disaster].copy()
print(f"{disaster} Events Shape:", df.shape)

flood Events Shape: (4274, 36)


In [40]:
def compute_duration(row):
    try:
        start_date = pd.Timestamp(year=int(row['Start Year']), month=int(row['Start Month']), day=int(row['Start Day']))
        end_date = pd.Timestamp(year=int(row['End Year']), month=int(row['End Month']), day=int(row['End Day']))
        return (end_date - start_date).days
    except Exception:
        return np.nan

df['duration'] = df.apply(compute_duration, axis=1)


# Aggregate Data by Grid Cell

---
Aggregate Data by Grid Cell (Flood Events Only):

Group the data by grid_id and compute the following parameters:

*total_event_count*: Count of flood events in the grid.

*earliest_event_year*: Minimum of the 'year' column.

*latest_event_year*: Maximum of the 'year' column.

*total_damages*: Sum of "Total Damages, Adjusted ('000 US$)".

*primary_admin1*: Most common administrative region (adm1).

*unique_locations*: Comma-separated list of unique location names.

*avg_duration*: Average duration (in days) of flood events.

#Additional parameters (if available):


*avg_damage_per_flood*: Total_damages divided by total_event_count.

*total_deaths*: Sum of "Total Deaths".

*total_affected*: Sum of "Total Affected".

*reconstruction_costs*: Sum of "Reconstruction Costs, Adjusted ('000 US$)".

*predominant_subtype*: Mode of "Disaster Subtype".

*ofda_response_count*: Count of non-null "OFDA Response".

*predominant_river_basin*: Mode of "River Basin".

*flood_recurrence_interval*: (latest_event_year - earliest_event_year)divided by total_event_count.


In [41]:
def safe_divide(numerator, denominator):
    return numerator / denominator if denominator and denominator != 0 else np.nan

In [42]:
# Aggregate all params
aggregated = df.groupby('grid_id').agg(
    count = ('disasterno', 'count'),
    first_year = ('year', 'min'),
    last_year = ('year', 'max'),
    total_damages = ("Total Damages, Adjusted ('000 US$)", 'sum'),
    avg_duration = ('duration', 'mean'),
    total_deaths = ('Total Deaths', 'sum') if 'Total Deaths' in df.columns else (lambda x: np.nan),
    total_affected = ('Total Affected', 'sum') if 'Total Affected' in df.columns else (lambda x: np.nan),
    predominant_subtype = ('Disaster Subtype', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan) if 'Disaster Subtype' in df.columns else (lambda x: np.nan),
    # ofda_response_count = ('OFDA Response', lambda x: x.notnull().sum()) if 'OFDA Response' in df.columns else (lambda x: np.nan),
    predominant_river_basin = ('River Basin', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan) if 'River Basin' in df.columns else (lambda x: np.nan),
).reset_index()


In [43]:
# Compute derived parameters:
aggregated['avg_damage_per_event'] = aggregated.apply(
    lambda row: safe_divide(row['total_damages'], row['count']),
    axis=1
)
# Flood recurrence formula last-first divided by number of events in the interval
aggregated['recurrence_interval'] = aggregated.apply(
    lambda row: safe_divide(row['last_year'] - row['first_year'], row['count']),
    axis=1
)

In [44]:
aggregated['event_summary'] = aggregated.apply(
    lambda row: f"{row['count']} events from {int(row['first_year'])} to {int(row['last_year'])}, total damages: {row['total_damages'] if not pd.isna(row['total_damages']) else 'N/A'} ('000 US$)",
    axis=1
)

In [45]:
aggregated

,grid_id,count,first_year,last_year,total_damages,avg_duration,total_deaths,total_affected,predominant_subtype,predominant_river_basin,avg_damage_per_event,recurrence_interval,event_summary
0,-10_-36,1,2004,2004,531.0,1.000000,28.0,2254.0,Riverine flood,NaN,531.000000,0.000000,"1 events from 2004 to 2004, total damages: 531..."
1,-10_-37,5,1967,2009,777910.0,4.400000,451.0,2110149.0,Riverine flood,NaN,155582.000000,8.400000,"5 events from 1967 to 2009, total damages: 777..."
2,-10_-66,1,1987,1987,0.0,NaN,25.0,20000.0,Riverine flood,NaN,0.000000,0.000000,"1 events from 1987 to 1987, total damages: 0.0..."
3,-10_124,1,2010,2010,0.0,3.000000,16.0,200.0,Flash flood,NaN,0.000000,0.000000,"1 events from 2010 to 2010, total damages: 0.0..."
4,-10_14,2,2004,2005,0.0,30.500000,1.0,12000.0,Riverine flood,"Kapacala, Cuanza, Okavango rivers",0.000000,0.500000,"2 events from 2004 to 2005, total damages: 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,9_77,1,1992,1992,351659.0,0.000000,51.0,500.0,Riverine flood,NaN,351659.000000,0.000000,"1 events from 1992 to 1992, total damages: 351..."
1832,9_78,1,2015,2015,0.0,4.000000,8.0,200.0,Flash flood,NaN,0.000000,0.000000,"1 events from 2015 to 2015, total damages: 0.0..."
1833,9_80,3,1981,2018,1354808.0,1.333333,212.0,396602.0,Flash flood,NaN,451602.666667,12.333333,"3 events from 1981 to 2018, total damages: 135..."
1834,9_81,3,1994,2014,0.0,18.000000,43.0,1807655.0,Riverine flood,NaN,0.000000,6.666667,"3 events from 1994 to 2014, total damages: 0.0..."


In [46]:
#show percentage of missing data
aggregated.isna().sum()/len(aggregated)

,0
grid_id,0.000000
count,0.000000
first_year,0.000000
last_year,0.000000
total_damages,0.000000
avg_duration,0.084423
total_deaths,0.000000
total_affected,0.000000
predominant_subtype,0.168301
predominant_river_basin,0.581155


In [21]:
#compute an overall 'hazard score' based on damage, deaths and frequency
df = aggregated.copy()

# Step 1: Min-max normalization (ignoring NaN)
df['count_norm'] = (df['count'] - df['count'].min()) / (df['count'].max() - df['count'].min())
df['deaths_norm'] = (df['total_deaths'] - df['total_deaths'].min()) / (df['total_deaths'].max() - df['total_deaths'].min())
df['damages_norm'] = (df['total_damages'] - df['total_damages'].min()) / (df['total_damages'].max() - df['total_damages'].min())

# Step 2: Handle missing values
df['count_present'] = ~df['count'].isna()
df['deaths_present'] = ~df['total_deaths'].isna()
df['damages_present'] = ~df['total_damages'].isna()

# Step 3: Determine dynamic weights
df['num_present'] = df[['count_present', 'deaths_present', 'damages_present']].sum(axis=1)

# Assign weights dynamically
df['count_weight'] = df['count_present'] / df['num_present']
df['deaths_weight'] = df['deaths_present'] / df['num_present']
df['damages_weight'] = df['damages_present'] / df['num_present']

# Fill NA with 0 for Norms
df['count_norm'] = df['count_norm'].fillna(0)
df['deaths_norm'] = df['deaths_norm'].fillna(0)
df['damages_norm'] = df['damages_norm'].fillna(0)

# Step 4: Compute severity score dynamically
df['severity_score'] = (
df['count_norm'] * df['count_weight'] +
df['deaths_norm'] * df['deaths_weight'] +
df['damages_norm'] * df['damages_weight'])

#drop those cols
df= df.drop(df.filter(regex='norm|weight|present').columns, axis=1)


df.sort_values(by='severity_score', ascending=False).head(50)
## TO DO: is min-max normalization the best?? -> @Luca can you think about this, using deaths, loss, count, to compute a severity score


,grid_id,count,first_year,last_year,total_damages,avg_duration,total_deaths,total_affected,predominant_subtype,predominant_river_basin,avg_damage_per_event,recurrence_interval,event_summary,severity_score
1038,32_117,21,1980,2015,121621935.0,24.833333,18549.0,1.033919e+09,Riverine flood,43710,5.791521e+06,1.666667,"21 events from 1980 to 2015, total damages: 12...",0.760331
1017,31_112,21,1980,2017,53587610.0,12.578947,4172.0,2.555336e+08,Riverine flood,Dalongtan Reservoir on Qingjiang River,2.551791e+06,1.761905,"21 events from 1980 to 2017, total damages: 53...",0.415149
446,10_-68,7,1994,2011,5440315.0,5.714286,30194.0,6.109770e+05,Riverine flood,"Guaire, Carmen de Uria, Caugaguita, Upire, Toc...",7.771879e+05,2.428571,"7 events from 1994 to 2011, total damages: 544...",0.414910
906,27_107,31,1988,2018,22364846.0,10.620690,1707.0,1.634767e+08,Riverine flood,Duliujiang,7.214466e+05,0.967742,"31 events from 1988 to 2018, total damages: 22...",0.413474
931,28_112,26,1985,2016,32389279.0,9.739130,4048.0,2.157741e+08,Riverine flood,Xiangijiang River,1.245742e+06,1.192308,"26 events from 1985 to 2016, total damages: 32...",0.411237
900,26_93,19,1968,2015,13193169.0,21.562500,3993.0,1.606086e+08,Riverine flood,Brahmaputra,6.943773e+05,2.473684,"19 events from 1968 to 2015, total damages: 13...",0.280241
894,26_86,12,1975,2013,9817307.0,12.333333,11172.0,1.621085e+08,Riverine flood,"Gandak, Kosi, Sone, Bagmati, Andhawara",8.181089e+05,3.166667,"12 events from 1975 to 2013, total damages: 98...",0.272465
1110,34_72,17,1976,2015,14340690.0,8.294118,3249.0,2.778529e+07,Flash flood,"Khatayan, Alingar",8.435700e+05,2.294118,"17 events from 1976 to 2015, total damages: 14...",0.252950
1771,7_80,22,1984,2018,625656.0,5.050000,612.0,3.469461e+06,Riverine flood,Kelani,2.843891e+04,1.545455,"22 events from 1984 to 2018, total damages: 62...",0.241804
1788,8_100,20,1975,2016,4183351.0,7.055556,1194.0,8.149182e+06,Riverine flood,"Kelantan, Lebir, Golok, Semerak, Tambatan Dira...",2.091675e+05,2.050000,"20 events from 1975 to 2016, total damages: 41...",0.235758


In [61]:
######################## NORMALIZATION  ################################

# Min-Max normalization is too sensitive to skewed values. Min-Max with z-score normalization is also inappropriate since our data is not normal

# Better to use log normalization for large values AND robust scaling


from sklearn.preprocessing import RobustScaler
import numpy as np

def compute_severity_score(df,
                           count_col='count',
                           deaths_col='total_deaths',
                           damages_col='total_damages',
                           weight_count=0.2,
                           weight_deaths=0.5,
                           weight_damages=0.3):
    """
    Applies log + robust scaling to the columns for event count, deaths, and damages.
    Then computes a severity_score based on the chosen weights:
        severity_score = scaled_count*weight_count + scaled_deaths*weight_deaths + scaled_damages*weight_damages
    Finally normalizes the result to 0-100 range and returns a DataFrame with two new columns:
        - severity_score
        - severity_score_normalized
    The original df is not modified in place.
    """
    df_out = df.copy()  # So we don't overwrite original

    # Replace NaN with 0 for these columns if needed
    df_out[count_col]   = df_out[count_col].fillna(0)
    df_out[deaths_col]  = df_out[deaths_col].fillna(0)
    df_out[damages_col] = df_out[damages_col].fillna(0)

    # Log transform: log(1 + x)
    df_out['count_log']   = np.log1p(df_out[count_col])
    df_out['deaths_log']  = np.log1p(df_out[deaths_col])
    df_out['damages_log'] = np.log1p(df_out[damages_col])

    # Apply robust scaling
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(df_out[['count_log','deaths_log','damages_log']])
    df_out['count_scaled']   = scaled_data[:,0]
    df_out['deaths_scaled']  = scaled_data[:,1]
    df_out['damages_scaled'] = scaled_data[:,2]

    # Compute severity score with user-specified weights
    df_out['severity_score'] = (
        df_out['count_scaled']  * weight_count   +
        df_out['deaths_scaled'] * weight_deaths  +
        df_out['damages_scaled']* weight_damages
    )

    # Normalize severity score to 0-100
    min_s = df_out['severity_score'].min()
    max_s = df_out['severity_score'].max()
    # add a tiny epsilon to avoid division by zero
    df_out['severity_score_normalized'] = (
        (df_out['severity_score'] - min_s)
        / (max_s - min_s + 1e-9) * 100
    )

    # Drop intermediate columns
    df_out.drop(columns=['count_log','deaths_log','damages_log',
                         'count_scaled','deaths_scaled','damages_scaled'], inplace=True)

    return df_out


# I checked some of the grid ids and they align with devastating flooding events

In [62]:
# Append 'flood_' to all column names except 'grid_id'
agg_flood = df.copy()
agg_flood = agg_flood.rename(columns=lambda col: f"{disaster}_{col}" if col != "grid_id" else col)

#merge with all_disaster data
agg_flood = pd.merge(agg_all, agg_flood, on='grid_id', how='left')

#output:
agg_flood.to_csv("ui/flood_grid_summary.csv")
# print("Aggregated flood grid summary data saved to 'flood_grid_summary.csv'")

In [67]:
agg_flood

,grid_id,all_count,all_damages,lat,lon,primary_admin1,unique_locations,all_deaths,predominant_disaster,all_disastertypes,...,flood_damages_scaled,flood_count_present,flood_deaths_present,flood_damages_present,flood_num_present,flood_count_weight,flood_deaths_weight,flood_damages_weight,flood_severity_score,flood_severity_score_normalized
0,-10_-36,1,531.0,-10,-36,Alagoas,Maceio,28.0,flood,flood,...,0.540498,True,True,True,3.0,0.333333,0.333333,0.333333,0.089563,25.080788
1,-10_-37,6,895928.0,-10,-37,Alagoas,Alagoas,511.0,flood,"flood, landslide",...,1.168062,True,True,True,3.0,0.333333,0.333333,0.333333,0.391912,59.824916
2,-10_-48,1,0.0,-10,-48,Tocantins,Tocantins,0.0,drought,drought,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-10_-66,1,0.0,-10,-66,Pando,Nueva Esperanza,25.0,flood,flood,...,0.000000,True,True,True,3.0,0.333333,0.333333,0.333333,0.030188,18.257785
4,-10_-75,2,0.0,-10,-75,Pasco,Pasco,418.0,extreme temperature,extreme temperature,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2822,9_78,1,0.0,9,78,Tamil Nadu,Virudhunagar district,8.0,flood,flood,...,0.000000,True,True,True,3.0,0.333333,0.333333,0.333333,-0.021546,12.312850
2823,9_79,2,0.0,9,79,Tamil Nadu,"Ramanathapurum, Rameswaram",507.0,storm,storm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2824,9_80,3,1354808.0,9,80,Kilinochchi,"Vavuniya, Kilinochchi district , Northern",212.0,flood,flood,...,1.215837,True,True,True,3.0,0.333333,0.333333,0.333333,0.321001,51.676309
2825,9_81,4,325863.0,9,81,Trincomalee,"Mullaitivu district, Trincomalee",249.0,flood,"flood, storm",...,0.000000,True,True,True,3.0,0.333333,0.333333,0.333333,0.122509,28.866845


In [ ]:
#@Luca: what is 'total affected feature'


In [29]:
# Answer:
#In the EMDAT dataset, 'Total Affected' is the estimated count of people who experienced at least one of the following:
#  - Injury
#  - Being left homeless
#  - Requiring immediate assistance (food, water, shelter, sanitation, etc.)

# See output of columns of emdat right before processing

In [ ]:

## TO DO: is min-max normalization the best?? -> @Luca can you think about this, using deaths, loss, count, to compute a severity score
# TO DO: replicate the process for other disasters.
# TO DO: merge other data sets with this, e.g. NLP raw data.
# TO DO: add live predictions for 1-5 year

In [63]:
################################# MERGE WITH NLP DESCRIPTIONS #############################

# 1) Read the original df_cls_transfer128.csv
df_nlp_cls = pd.read_csv('data/nlp/df_cls_transfer128.csv')

# 2) Define a helper function to convert '( lat, lon )' to 'lat_lon'
def tuple_to_str(t):
    """Convert a tuple-like string '( -10, -36 )' into '-10_-36' format."""
    t = t.strip()  # remove extra spaces
    t = t.replace('(', '').replace(')', '')  # remove parentheses
    parts = [x.strip() for x in t.split(',')]
    if len(parts) == 2:
        return f"{parts[0]}_{parts[1]}"
    else:
        return t  # fallback if something unexpected

# 3) Convert 'grid_id' in df_nlp_cls to match the style of 'agg_flood'
df_nlp_cls['grid_id'] = df_nlp_cls['grid_id'].astype(str).apply(tuple_to_str)

# 4) Subset to just 'grid_id' and 'txt' columns
df_nlp_text = df_nlp_cls[['grid_id', 'txt']].copy()

# 5) Merge with 'agg_flood' on 'grid_id' only
agg_flood_text_merged = pd.merge(
    agg_flood,    # your existing aggregated flood DataFrame
    df_nlp_text,  # the text data
    on='grid_id',
    how='left'
)

print("Merged shape:", agg_flood_text_merged.shape)


Merged shape: (2827, 39)


In [74]:
agg_flood_text_merged

,grid_id,all_count,all_damages,lat,lon,primary_admin1,unique_locations,all_deaths,predominant_disaster,all_disastertypes,...,flood_count_present,flood_deaths_present,flood_damages_present,flood_num_present,flood_count_weight,flood_deaths_weight,flood_damages_weight,flood_severity_score,flood_severity_score_normalized,txt
0,-10_-36,1,531.0,-10,-36,Alagoas,Maceio,28.0,flood,flood,...,True,True,True,3.0,0.333333,0.333333,0.333333,0.089563,25.080788,NaN
1,-10_-37,6,895928.0,-10,-37,Alagoas,Alagoas,511.0,flood,"flood, landslide",...,True,True,True,3.0,0.333333,0.333333,0.333333,0.391912,59.824916,NaN
2,-10_-48,1,0.0,-10,-48,Tocantins,Tocantins,0.0,drought,drought,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-10_-66,1,0.0,-10,-66,Pando,Nueva Esperanza,25.0,flood,flood,...,True,True,True,3.0,0.333333,0.333333,0.333333,0.030188,18.257785,NaN
4,-10_-75,2,0.0,-10,-75,Pasco,Pasco,418.0,extreme temperature,extreme temperature,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2822,9_78,1,0.0,9,78,Tamil Nadu,Virudhunagar district,8.0,flood,flood,...,True,True,True,3.0,0.333333,0.333333,0.333333,-0.021546,12.312850,NaN
2823,9_79,2,0.0,9,79,Tamil Nadu,"Ramanathapurum, Rameswaram",507.0,storm,storm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2824,9_80,3,1354808.0,9,80,Kilinochchi,"Vavuniya, Kilinochchi district , Northern",212.0,flood,flood,...,True,True,True,3.0,0.333333,0.333333,0.333333,0.321001,51.676309,NaN
2825,9_81,4,325863.0,9,81,Trincomalee,"Mullaitivu district, Trincomalee",249.0,flood,"flood, storm",...,True,True,True,3.0,0.333333,0.333333,0.333333,0.122509,28.866845,NaN


# Replicate for other disasters

In [70]:
def replicate_aggregator(merged_df, disaster, all_ids=None):
    """
    1) Partially matches `disastertype` (contains `disaster`) so e.g. 'flood'
       also gets 'flood, landslide'.
    2) Groups by grid_id to get: count, first_year, last_year, total_damages,
       total_deaths, total_affected, avg_duration.
    3) Merges on `all_ids` to keep zero-count rows (matching your original
       big table shape).
    4) Uses your log+robust scaling for severity (compute_severity_score).
    5) Renames columns to <disaster>_ prefix, except 'grid_id'.
    """

    # 1) Filter with partial match for the given disaster
    df_dis = merged_df[
        merged_df['disastertype'].str.lower().str.contains(disaster.lower(), na=False)
    ].copy()

    # 2) Optionally compute an event duration
    def compute_duration(row):
        try:
            start_date = pd.Timestamp(
                year=int(row['Start Year']),
                month=int(row['Start Month']),
                day=int(row['Start Day'])
            )
            end_date = pd.Timestamp(
                year=int(row['End Year']),
                month=int(row['End Month']),
                day=int(row['End Day'])
            )
            return (end_date - start_date).days
        except:
            return np.nan

    df_dis['duration'] = df_dis.apply(compute_duration, axis=1)

    # 3) Group by grid_id
    grouped = df_dis.groupby('grid_id').agg(
        count=('disasterno','count'),
        first_year=('year','min'),
        last_year=('year','max'),
        total_damages=("Total Damages, Adjusted ('000 US$)", 'sum'),
        total_deaths=('Total Deaths','sum'),
        total_affected=('Total Affected','sum'),
        avg_duration=('duration','mean')
    ).reset_index()

    # If we want to preserve the full set of grid IDs (including zero-count),
    # we merge on all_ids
    if all_ids is not None:
        grouped = pd.merge(all_ids, grouped, on='grid_id', how='left')
        # Fill missing with 0 (or other sentinel) where appropriate
        for col in ['count','total_damages','total_deaths','total_affected','avg_duration']:
            if col in grouped.columns:
                grouped[col] = grouped[col].fillna(0)
        for col in ['first_year','last_year']:
            if col in grouped.columns:
                grouped[col] = grouped[col].fillna(-9999)

    # 4) Apply your log+robust severity approach:
    #    Make sure you have defined `compute_severity_score` earlier in your code.
    grouped = compute_severity_score(
        grouped,
        count_col='count',
        deaths_col='total_deaths',
        damages_col='total_damages',
        weight_count=0.2,
        weight_deaths=0.5,
        weight_damages=0.3
    )

    # 5) Rename columns to <disaster>_ prefix
    grouped = grouped.rename(columns={
        c: f"{disaster}_{c}" for c in grouped.columns if c != 'grid_id'
    })

    return grouped

# ------------------- EXAMPLE USAGE -------------------
# Suppose 'merged_df' is your joined EMDAT+GDIS data.
# We create 'all_ids' to keep all grid_ids for merges (like your original aggregator).
all_ids = pd.DataFrame({'grid_id': merged_df['grid_id'].unique()})

# Now replicate for each disaster with "standard" aggregator names.
agg_flood       = replicate_aggregator(merged_df, 'flood', all_ids=all_ids)
agg_earthquake  = replicate_aggregator(merged_df, 'earthquake', all_ids=all_ids)
agg_storm       = replicate_aggregator(merged_df, 'storm', all_ids=all_ids)
agg_drought     = replicate_aggregator(merged_df, 'drought', all_ids=all_ids)

print("Shapes:")
print("  agg_flood:", agg_flood.shape)
print("  agg_earthquake:", agg_earthquake.shape)
print("  agg_storm:", agg_storm.shape)
print("  agg_drought:", agg_drought.shape)

# Optionally, show a few rows for flood:
# display(agg_flood.head(5))

Shapes:
  agg_flood: (2827, 10)
  agg_earthquake: (2827, 10)
  agg_storm: (2827, 10)
  agg_drought: (2827, 10)


In [71]:
agg_earthquake

,grid_id,earthquake_count,earthquake_first_year,earthquake_last_year,earthquake_total_damages,earthquake_total_deaths,earthquake_total_affected,earthquake_avg_duration,earthquake_severity_score,earthquake_severity_score_normalized
0,42_19,1.0,1979.0,1979.0,0.0,0.0,80.0,0.0,0.138629,1.175207
1,-17_16,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,22_91,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,24_90,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,22_92,1.0,2000.0,2000.0,0.0,0.0,1000.0,0.0,0.138629,1.175207
...,...,...,...,...,...,...,...,...,...,...
2822,40_-9,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2823,25_52,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2824,11_34,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2825,-5_142,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [72]:
agg_storm

,grid_id,storm_count,storm_first_year,storm_last_year,storm_total_damages,storm_total_deaths,storm_total_affected,storm_avg_duration,storm_severity_score,storm_severity_score_normalized
0,42_19,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.00,0.000000,0.000000
1,-17_16,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.00,0.000000,0.000000
2,22_91,10.0,1966.0,2013.0,6699616.0,156176.0,24712993.0,1.50,9.720360,100.000000
3,24_90,13.0,1967.0,2006.0,36367.0,1524.0,293263.0,0.25,6.555553,67.441459
4,22_92,2.0,1997.0,2009.0,0.0,118.0,3071947.0,0.50,2.040697,20.994046
...,...,...,...,...,...,...,...,...,...,...
2822,40_-9,1.0,2018.0,2018.0,124636.0,2.0,88.0,2.00,4.116189,42.346053
2823,25_52,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.00,0.000000,0.000000
2824,11_34,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.00,0.000000,0.000000
2825,-5_142,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.00,0.000000,0.000000


In [73]:
agg_drought

,grid_id,drought_count,drought_first_year,drought_last_year,drought_total_damages,drought_total_deaths,drought_total_affected,drought_avg_duration,drought_severity_score,drought_severity_score_normalized
0,42_19,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,-17_16,1.0,2017.0,2017.0,0.0,0.0,1420000.0,0.0,0.138629,1.413249
2,22_91,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,24_90,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,22_92,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
2822,40_-9,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2823,25_52,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2824,11_34,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2825,-5_142,0.0,-9999.0,-9999.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [ ]:
#### ---- old codes ------

## There is a lot of missing data for $ costs; Replacing 0s with NaNs

In [ ]:
# Replace zeros with NaN for cost-related columns in the aggregated DataFrame.
# This assumes that a 0 value for damages or reconstruction costs should be treated as missing data.
cost_columns = ['total_damages', 'reconstruction_costs']
for col in cost_columns:
    aggregated[col] = aggregated[col].replace(0, np.nan)

# Recompute the derived parameter 'avg_damage_per_flood' after the replacement.
aggregated['avg_damage_per_flood'] = aggregated.apply(
    lambda row: safe_divide(row['total_damages'], row['total_event_count']),
    axis=1
)

# Select and Order Final Columns

## We now keep only the desired columns:

 1. grid_id  
 2. central_lat  
 3. central_lon  
 4. total_event_count  
 5. earliest_event_year  
 6. latest_event_year  
 7. total_damages  
 8. primary_admin1  
 9. unique_locations  
 10. avg_duration  
 11. avg_damage_per_flood  
 12. total_deaths  
 13. total_affected  
 14. reconstruction_costs  
 15. predominant_subtype  
 16. ofda_response_count  
 17. predominant_river_basin  
 18. flood_recurrence_interval
 19. event_summary

In [ ]:
final_columns = ['grid_id', 'lat', 'lon', 'total_event_count',
                 'earliest_event_year', 'latest_event_year',
                 'total_damages', 'primary_admin1', 'unique_locations', 'avg_duration',
                 'avg_damage_per_flood', 'total_deaths', 'total_affected',
                 'reconstruction_costs', 'predominant_subtype', 'ofda_response_count',
                 'predominant_river_basin', 'flood_recurrence_interval', 'event_summary']

final_df = aggregated[final_columns].copy()


grid_id                      0.000000
central_lat                  0.000000
central_lon                  0.000000
total_event_count            0.000000
earliest_event_year          0.000000
latest_event_year            0.000000
total_damages                0.507625
primary_admin1               0.000000
country                      0.984205
unique_locations             0.000000
avg_duration                 0.084423
avg_damage_per_flood         0.507625
total_deaths                 0.000000
total_affected               0.000000
reconstruction_costs         0.994009
predominant_subtype          0.168301
ofda_response_count          0.000000
predominant_river_basin      0.581155
flood_recurrence_interval    0.000000
event_summary                0.000000
dtype: float64

,grid_id,count,first_year,last_year,total_damages,avg_duration,total_deaths,total_affected,predominant_subtype,predominant_river_basin,avg_damage_per_event,recurrence_interval,event_summary,severity_score
1038,32_117,21,1980,2015,121621935.0,24.833333,18549.0,1.033919e+09,Riverine flood,43710,5.791521e+06,1.666667,"21 events from 1980 to 2015, total damages: 12...",0.760331
1017,31_112,21,1980,2017,53587610.0,12.578947,4172.0,2.555336e+08,Riverine flood,Dalongtan Reservoir on Qingjiang River,2.551791e+06,1.761905,"21 events from 1980 to 2017, total damages: 53...",0.415149
446,10_-68,7,1994,2011,5440315.0,5.714286,30194.0,6.109770e+05,Riverine flood,"Guaire, Carmen de Uria, Caugaguita, Upire, Toc...",7.771879e+05,2.428571,"7 events from 1994 to 2011, total damages: 544...",0.414910
906,27_107,31,1988,2018,22364846.0,10.620690,1707.0,1.634767e+08,Riverine flood,Duliujiang,7.214466e+05,0.967742,"31 events from 1988 to 2018, total damages: 22...",0.413474
931,28_112,26,1985,2016,32389279.0,9.739130,4048.0,2.157741e+08,Riverine flood,Xiangijiang River,1.245742e+06,1.192308,"26 events from 1985 to 2016, total damages: 32...",0.411237
900,26_93,19,1968,2015,13193169.0,21.562500,3993.0,1.606086e+08,Riverine flood,Brahmaputra,6.943773e+05,2.473684,"19 events from 1968 to 2015, total damages: 13...",0.280241
894,26_86,12,1975,2013,9817307.0,12.333333,11172.0,1.621085e+08,Riverine flood,"Gandak, Kosi, Sone, Bagmati, Andhawara",8.181089e+05,3.166667,"12 events from 1975 to 2013, total damages: 98...",0.272465
1110,34_72,17,1976,2015,14340690.0,8.294118,3249.0,2.778529e+07,Flash flood,"Khatayan, Alingar",8.435700e+05,2.294118,"17 events from 1976 to 2015, total damages: 14...",0.252950
1771,7_80,22,1984,2018,625656.0,5.050000,612.0,3.469461e+06,Riverine flood,Kelani,2.843891e+04,1.545455,"22 events from 1984 to 2018, total damages: 62...",0.241804
1788,8_100,20,1975,2016,4183351.0,7.055556,1194.0,8.149182e+06,Riverine flood,"Kelantan, Lebir, Golok, Semerak, Tambatan Dira...",2.091675e+05,2.050000,"20 events from 1975 to 2016, total damages: 41...",0.235758


# Save data to csv

In [ ]:
final_columns = ['grid_id', 'lat', 'lon', 'total_event_count',
                 'earliest_event_year', 'latest_event_year',
                 'total_damages', 'primary_admin1', 'unique_locations', 'avg_duration',
                 'avg_damage_per_flood', 'total_deaths', 'total_affected',
                 'reconstruction_costs', 'predominant_subtype', 'ofda_response_count',
                 'predominant_river_basin', 'flood_recurrence_interval', 'event_summary']

final_df = df.copy()

final_df.to_csv('/content/drive/MyDrive/mml_flood/UI/flood_grid_summary.csv', index=False)
print("Aggregated flood grid summary data saved to '/content/drive/MyDrive/mml_flood/UI/flood_grid_summary.csv'.")

Aggregated flood grid summary data saved to '/content/drive/MyDrive/mml_flood/UI/flood_grid_summary.csv'.


In [75]:
!git status

Refresh index: 100% (26/26), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ui/data_processing_summary.ipynb
	deleted:    ui/flood_grid_summary.csv
	deleted:    ui/flood_grid_summary_by_year.csv

no changes added to commit (use "git add" and/or "git commit -a")
